In [6]:
from leaderboard import *


## Alle Parts 2 (Sekunden)

In [7]:
data = load_data(leaderboard_id=976545, year=2023)
data


,1,2,3,4,5,6,7,8
2281794,<NA>,1049,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3309362,4470,734,1587,1333,7970,393,977,3333
3376741,1548,<NA>,<NA>,2705,<NA>,<NA>,<NA>,<NA>
3lacki,18222,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
740740,7287,39773,1965,2145,<NA>,<NA>,<NA>,<NA>
999754,6110,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
@emioderso,14007,1929,<NA>,<NA>,<NA>,429,<NA>,<NA>
attoPascal,1592,295,425,1525,4403,538,26905,1128
Bene-dykt,30,329,3871,<NA>,<NA>,<NA>,<NA>,<NA>
bieroeffner,236959,2743,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


## Part 2 Leaderboard

In [8]:
points(data).head(20)


,pts,name
1,415,Vonuras
2,404,JustAlittleWolf
3,402,bu19akov
4,392,msonnberger
5,368,attoPascal
6,367,memoriesadrift
7,357,zhekemist
8,343,drizzelat
9,335,NotsoblackReaper
10,331,reputation


## Sekunden für Day 8, Part 2

In [9]:
seconds_latest_day(data)


,8
bu19akov,617
msonnberger,964
attoPascal,1128
zhekemist,1491
Vonuras,1581
JustAlittleWolf,1800
memoriesadrift,1806
fwoerister,2482
3309362,3333
clemy,5966


## Minuten für alle Parts 2

In [10]:
sum_seconds(data) // 60


,0
bu19akov,144
Vonuras,170
zhekemist,257
memoriesadrift,276
JustAlittleWolf,315
3309362,346
msonnberger,513
clemy,525
Salominius,569
attoPascal,613
